# Team 7UP

## SDS Datathon 2024 Champion's Group

### By Winston, Maximus, Jian Hao, Caleb, Ryan

##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
#%pip install pandas 
#%pip install matplotlib
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install scipy
%pip install xgboost
%pip install kmodes

In [2]:
# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .csv file inside that. A relative path *must* be used when loading data into pandas

In [ ]:
filepath = "./data/catA_train.csv" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

df = pd.read_csv(filepath)
df2 = df.copy()

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

### **Part 0: Background Information**

**0.1. General Information**

In [ ]:
print(df.info())

**0.2. Unique Values**

In [ ]:
# Total number of unique values for each column
print(df.nunique())

In [ ]:
# Unique value in Company Status (Active/Inactive)
print(df['Company Status (Active/Inactive)'].unique())

In [ ]:
# Number of occurences of unique values in Ownership Type
print(df["Ownership Type"].value_counts())

**0.3. Modal Values**

In [ ]:
# Percentage occurrence of modal values for each column
pd.DataFrame({'Columns': df.columns, 'Val (%)': [((df[x].isin(df[x].mode()).sum() / df.shape[0]) * 100).round(1)  for x in df]})

In [ ]:
# pd.DataFrame({'Columns': df.columns, 'Val':[df[x].isin(df[x].mode()).sum() for x in df]})

value_counts = df['Parent Company'].value_counts()

values_appear_once = value_counts[value_counts <= 1]
print(values_appear_once)
# count_values_appear_once = len(values_appear_once)
# print("Number of values appearing only once:", count_values_appear_once)

**0.4. Missing Values**

In [ ]:
# Total number of missing values for each column
print(df.isna().sum())

**0.5. Correlation Matrix for Variables**



In deriving the correlation matrix, we are required to: 

* Perform imputation for missing values.

* Perform categorical encoding to convert non-numerical data to numerical data.

(Why one-hot encoding over label encoding, or otherwise?)

In [ ]:
# Imputation for missing values.

# Perform one-hot encoding
df = pd.get_dummies(df, columns=['Entity Type'], prefix = 'Entity Type')
df = pd.get_dummies(df, columns=['Parent Country'], prefix='Parent Country')

### **Part 1: Data Cleaning**

Data cleaning involves identifying and rectifying errors and inconsistencies to enhance the overall quality and trustworthiness of the data. This is pivotal in improving model performance.

We perform the following processes as part of data cleaning: 
* **1. Remove irrelevant columns.**
* **2. Remove irrelevant rows.**

    The above removal processes are important as they ensure subsequent analysis is not slowed down or confused by irrelevant data.



The following processes are also under Part 1 but have been done in 0.4. as they are instrumental in deriving the correlation matrix. Corresponding justifications are also provided in 0.4.

* **Convert non-numerical data to numerical form.**
    
    This is important as it ensures machine models subsequently employed are able to interpret the data used, given these models can only interpret numerical values.

* **Impute missing data with relevant values.**

    This is important in ensuring completeness of datasets, reducing biasedness and inefficiency of statistical models.


**1.1. Remove Irrelevant Columns**

We remove columns with data deemed irrelevant due to the presence of the following:

  * **1.1.1. Personally Identifiable Information (PII)**
  
    We remove the 2 columns for the attributes `AccountID` and `Company` as they constitute PII. PII is considered irrelevant in the case of sales forecasting because the focus is on aggregated and anonymized data rather than individual-level details. Sales forecasting involves analyzing trends, patterns, and historical data to make predictions about future sales performance at a broader level.
    


  * **1.1.2. Redundant Descriptions**

    We remove the 3 columns for the attributes **`8-Digit SIC Description`**, **`Industry`** and **`Company Description`** as they constitute redundant descriptions. 
    
    `8-Digit SIC Description` and `Industry` are deemed redundant as they merely provide further information on `SIC Code` (which is already part of the DataFrame). We choose to retain `SIC Code` rather than the descriptions as `SIC Code` constitutes numerical data which is more easily interpreted by machine learning models.
    
    `Company Description` is deemed redundant as it merely provides further information on Company (which has already been removed in 1.1.1.).

  * **1.1.3. Insufficient Variability**

    We remove the 2 columns for the attributes **`Company Status (Active/Inactive)`** and **`Ownership Type`** as the values are not sufficiently variable. Hence, the 2 columns do not provide valuable information for further analysis.
    
    In `Company Status (Active/Inactive)`, the value in each row is constant (i.e. "Active", according to 0.2.).

    In `Ownership Type`, 96.8% of the values are "Private" (according to 0.2. and 0.3.), which suggests relatively limited variability. Moreover, none of the values rank in the top-quartile (i.e. top ...) when it comes to correlation with the target variable (Sales (Domestic Ultimate Total USD)). Hence, there is little ground to retain this column in the spirit of attempting to reduce the complexity of the model (prevent overfitting).
    
    

    

  * **1.1.4. Excessive Missing Information**

    We remove the column **`Square Footage`** as the data fields associated with the attribute are all empty (number of unique values in the column is 0 according to 0.2.). Thus, the column does not provide any information for analysis.

  * **1.1.5. Excessive Number of Unique Values with Clustering Difficulties**

    Columns of data with high cardinality (i.e. contain many unique values) should be considered for removal, in order to improve model performance. High cardinality in data results in sparser representation of each category in the dataset, which makes estimates of relationships between features and target variable less stable. 
    
    Moreover, in the presence of high cardinality, overfitting may occur (i.e. model performs well on the training set but poorly on new data), as the model may memorize the training data and capturing specific instances rather than learning generalizable patterns. 

    We identify **`Parent Company`**, **`Global Ultimate Company`** and **`Domestic Ultimate Company`** as columns with high cardinality. Clustering to reduce cardinality is difficult as the companies represented in the columns are 17,882, 14,145 and 15,597 individual entities respectively (according to 0.2.) with a wide range of different characteristics. Hence, we decide to remove the above 3 columns.



* **1.1.6. Multicollinearity**

    When there is multicollinearity (i.e. 2 attributes have a high degree of correlation), one of the attributes should be considered for removal. This is because the information from the second attribute would be redundant should the first attribute remain in the dataset. Having redundant information in the model would not contribute significantly to predicting sales figures and could also lead to instability. 
    
    Furthermore, high correlation between 2 attributes makes it difficult to interpret the individual impact of each attribute.

    We define the presence of multicollinearity as the existence of an absolute(-valued) Pearson correlation coefficient of at least 0.8. Applying this criterion, we identify that `SIC Code` and `8-Digit SIC Code` are highly correlated, as shown in the code below. 
    
    With the above in mind, we choose to remove the column **`8-Digit SIC Code`** rather than `SIC Code`. This is due to 2 reasons. Firstly, the `8-Digit SIC Code` is less correlated with domestic sales figures (i.e. correlation coefficient is of smaller magnitude, 1). Secondly, there are more unique values for `8-Digit SIC Code`, which entails thie issues of high cardinality discussed in 1.1.5.

    Furthermore, we will set a threshold on the SIC codes, only leaving in those that have more than 7 occurences, but labeling the rest as "Others".

In [ ]:
from scipy.stats import pearsonr

# Calculate the Pearson's correlation coefficient
correlation, p_value = pearsonr(df2['SIC Code'], df2['8-Digit SIC Code'])

# Print the correlation coefficient
print(f"Pearson's correlation coefficient: {correlation}")

In [ ]:
# Find the frequency of each industry
sic_code_frequency = df['SIC Code'].value_counts()

# Set a threshold for low-frequency SIC Codes
threshold = 7  # Adjust this threshold based on your preference

# Identify SIC Codes with frequency below the threshold
low_frequency_sic_codes = sic_code_frequency[sic_code_frequency < threshold].index

# Replace these low-frequency SIC Codes with a common label "Others"
df['SIC Code'] = df['SIC Code'].replace(low_frequency_sic_codes, 'Others')

sic_code_frequency1 = df['SIC Code'].value_counts()

# Display the updated DataFrame
print(sic_code_frequency1)

# Convert SIC Code to a 'category' data type so that it can be read by xgboost
df['SIC Code'] = df['SIC Code'].astype('category')

* **1.1.7. Domain Knowledge**

    We remove the column **`Fiscal Year End`** due to the following: We note that the objective is to predict annual sales figures, which implies all 12 months in a year will be evenly represented in the figure, regardless of when the fiscal year ends. Hence, the Fiscal Year End is irrelevant to the analysis.

* **1.1.8. Low Correlation to Target Variable** (or some feature selection to reduce dimensionality?)

    With the understanding that correlation does not imply causation, we have chosen to remove columns in parts 1.1.1 to 1.1.7., regardless of the degree of correlation to the target variable, based on the justifications which have been laid out. 

    The attributes `Latitude`, `Longitude`, `SIC Code`, `Entity Type`, `Parent Country`, `Employees Domestic`, `Sales Domestic`, `Sales Global`, `Is Domestic Ultimate` will be included.

    We seek here to remove ...

In [ ]:
columns_to_remove = ["AccountID", "Company", "8-Digit SIC Code", "8-Digit SIC Description", "Industry", "Company Description", "Company Status (Active/Inactive)", "Ownership Type", "Square Footage", "Parent Company", "Domestic Ultimate Company", "Global Ultimate Company", "Year Found", "Fiscal Year End", "Employees (Single Site)", "Employees (Global Ultimate Total)", "Import/Export Status", "Global Ultimate Country", "Is Global Ultimate"]
df = df.drop(columns=columns_to_remove)

**1.2. Remove Irrelevant Rows**

Having decided the columns to retain for subsequent analysis, we remove rows with data deemed irrelevant due to them containing:

* **1.2.1. Missing Data and Imputation is Deemed Unnecessary**

    We remove the rows with missing values for the `Latitude` and `Longitude` attributes. 

    We make this decision as we deem that rows for which `Latitude` and `Longitude` values are missing make up an insignificant part of the dataset. Exactly, they make up only 120 out of the 29,182 rows (according to 0.1. and 0.4.) (or 0.41% of the dataset). On the other hand, imputation would require that we derive the relative location of each of the entities and approximate the `Latitude` and `Longitude`, which, considering the significance to the dataset, is an unnecessarily cumbersome affair.

    Additionally, we also remove the rows with missing values in the `Employees (Domestic Ultimate Total)` column. This is because there are 79 missing values out of the total 29,182 (which accounts for only 0.27% of the dataset).

In [ ]:
# Remove rows without either Latitude or Longitude coordinates
df = df.dropna(subset=["LATITUDE", "LONGITUDE"])
df = df.dropna(subset=["Employees (Domestic Ultimate Total)"])

* **1.2.2. Missing Data Unsuitable for Imputation (Imputation doesn't make sense)**

    When we are unable to impute to deal with missing data in certain rows, the analysis with these rows will necessarily be incomplete. These rows should hence be removed to allow for complete analysis.





* **1.2.3. Outliers**

    From the columns deemed relevant, perform linear regression or some statistical method to derive outliers. Conclude that outliers make up an insignificant part of dataset.

### **Part 2: Exploratory Data Analysis (EDA)**

EDA is all about making sense of data in hand, before getting them dirty with any formal statistical analysis or modeling. It's a crucial step to understand the various aspects of the data you're working with, including the distribution, trends, and patterns that may exist. This process may include:

* Data Summarization: Descriptive statistics like mean, median, mode, standard deviation, etc., are used to summarize the data.
* Handling Missing Data and Outliers: EDA also involves identifying and dealing with missing data points or outliers that could skew the analysis.
* Visualization: Graphical techniques, including histograms, box plots, scatter plots, etc., are employed extensively. These help in visualizing the distribution of the data, relationships between variables, and identifying patterns or anomalies.
* Identifying Patterns and Relationships: EDA often involves looking for correlations, trends, and patterns that can lead to meaningful insights.


We will make use of a correlation matrix to observe how a numerical variable is affected by the other numerical variables in the dataset.

In [ ]:
# use of a correlation matrix to observe how variables in our dataset may be affected by others
numerical_df = df2.select_dtypes(include=['float64', 'int64'])

# Create a correlation matrix
correlation_matrix = numerical_df.corr()

# Assuming correlation_matrix is your correlation matrix
correlation_df = pd.DataFrame(correlation_matrix)

# Print or use the correlation DataFrame as needed
correlation_df

**2.1. Data Summarization**

This part has been covered under **0.1. General Information**

**2.2. Outlier Analysis**

We will be performing outlier analysis on `Sales Domestic` to identify anomalous data and remove them from our dataset. This is aimed at improving the accuracy of our model.

We will use a boxplot to help us identify the anomalous data. Data points that are more than 3 interquartile range to the left of quartile 1 or right of quartile 3 will be considered as outliers and removed from our dataset

In [ ]:
# Calculate the IQR
sales_col = 'Sales (Domestic Ultimate Total USD)'
Q1 = df[sales_col].quantile(0.25)
Q3 = df[sales_col].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

# Extract outliers
outliers = df[(df[sales_col] < lower_bound) | (df[sales_col] > upper_bound)]

# Remove rows with outliers
df = df[~((df[sales_col] < lower_bound) | (df[sales_col] > upper_bound))]

#Plot boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x=df[sales_col], color='skyblue', width=0.5, linewidth=2)
plt.title('Boxplot of Sales', fontsize=16)
plt.xlabel('Sales (Domestic Ultimate Total USD)', fontsize=14)
plt.ylabel('Sales', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

**2.3. Data Visualization and Identifying Relationships**

We will be performing EDA on `Latitude`, `Longitude`, `SIC Code`, `Entity Type`, `Parent Country`, `Employees Domestic`, `Sales Domestic`, `Sales Global`, `Is Domestic Ultimate`, which are the attributes we intend to keep, as outlined in 1.1.8:

* **2.3.1 `Latitude` & `Longitude`**

    We will perform a scatter plot to visualise coordinate data against sales.

In [ ]:
# import required libraries
import matplotlib.pyplot as plt

# we will need this for the color gradient representing how high the domestic sales figure is.
from matplotlib.colors import LogNorm
import seaborn as sns

In [ ]:
# for the upcoming geospatial analysis portion, we will use a small subset of the data provided for plotting.
filtered_df = df2[df2['SIC Code'].isin([5099, 1611, 8711, 4789])]

filtered_df.head(2)

In [ ]:
# Use LogNorm for more sensitivity to numerical changes
norm = LogNorm()

# Define a constant color (e.g., blue)
constant_color = 'blue'

# Define a color palette based on unique industries
industry_palette = sns.color_palette('Set1', n_colors=len(filtered_df['Industry'].unique()))

# Create a dictionary to map Industry to a unique color
industry_colors = dict(zip(filtered_df['Industry'].unique(), industry_palette))

# Scatter plot with varying color by Industry and varying alpha based on Sales values
scatter = plt.scatter(filtered_df['LONGITUDE'], filtered_df['LATITUDE'], c=filtered_df['Industry'].map(industry_colors), s=3, alpha=norm(filtered_df['Sales (Domestic Ultimate Total USD)']))

# Add a legend for Industry
legend_labels = filtered_df['Industry'].unique()
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=industry,
                              markerfacecolor=industry_colors[industry], markersize=4) for industry in legend_labels]
plt.legend(handles=legend_handles, title='Industry', prop={'size': 5}, loc='upper left', bbox_to_anchor=(1, 1))

# Set labels and title
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter Plot with Varying Color by Industry and Varying Alpha Based on Sales (LogNorm)')

# Show the plot
plt.show()

In [ ]:
# Use LogNorm for more sensitivity to numerical changes
norm = LogNorm()

# Define a colormap transitioning from blue to red
cmap = 'plasma'

# Scatter plot with colors based on Sales values and LogNorm
scatter = plt.scatter(filtered_df['LONGITUDE'], filtered_df['LATITUDE'], c=filtered_df['Sales (Domestic Ultimate Total USD)'], cmap=cmap, s=5, norm=norm)

# Add a colorbar
cbar = plt.colorbar(scatter, label='Sales')

# Set labels and title
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter Plot with Colors Based on Sales (LogNorm)')

# Show the plot
plt.show()

* **2.3.2 `SIC Code`**

    Here we plot a bar graph to visualise the SIC codes by sales.

In [ ]:
# Sort the data by revenue in descending order
sales_by_sic = df.groupby('SIC Code')['Sales (Domestic Ultimate Total USD)'].sum().sort_values(ascending=False)

# Select the top 10 SIC codes
top_10_sic = sales_by_sic.head(10)

# Plot the data
plt.figure(figsize=(12, 6))
top_10_sic.plot(kind='bar')
plt.xlabel('SIC Code')
plt.ylabel('Sales (Domestic Ultimate Total USD)')
plt.title('Top 10 Sales by SIC Code')
plt.xticks(rotation=45)
plt.show()

We see that the companies with different SIC Codes do have varying amounts of sales, therefore we keep this attribute.

* **2.3.3 `Entity Type`**

    Here we will perform a boxplot and a ANOVA test to understand the different entity types better, and to visualise the correlation between `Entity Type` and `Sales (Domestic Ultimate Total USD)`.

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Entity Type', y='Sales (Domestic Ultimate Total USD)', data=df2)
plt.xticks(rotation=90)  # Rotate x-axis labels for readability
plt.title('Entity Type vs Sales')
plt.show()

From the above boxplot, we can see that the variance and maximum values increase in the order of 'Branch', 'Independent', 'Parent', 'Subsidiary'.

In [ ]:
from scipy import stats

# Get a list of entity types
entity_types = df2['Entity Type'].unique()

# Create a list of sales for each entity type
sales_by_entity_type = [df2.loc[df2['Entity Type'] == entity_type, 'Sales (Domestic Ultimate Total USD)'] for entity_type in entity_types]

# Perform ANOVA test
f_val, p_val = stats.f_oneway(*sales_by_entity_type)
print(f"F-value: {f_val}, p-value: {p_val}")

The interpretation of the ANOVA test is as shown below.

The F-value of 14.80725389236939 is a measure of how much the means of `Sales (Domestic Ultimate Total USD)` for each `Entity Type` vary between the groups compared to how much they vary within the groups. A high F-value indicates that the means are significantly different.

The p-value of 1.2478331694905712e-09 is very small, much less than the typical threshold of 0.05. This indicates strong evidence against the null hypothesis. The null hypothesis for an ANOVA test is that all means are equal. Therefore, you can reject the null hypothesis and conclude that there is a statistically significant difference in the means of `Sales (Domestic Ultimate Total USD)` between the different `Entity Type`s.

In other words, the type of entity appears to have a significant effect on sales. 

* **2.3.4 `Parent Country`**

    Here we will perform a barplot and a ANOVA test to understand the different parent country better, and to visualise the correlation between `Parent Country` and `Sales (Domestic Ultimate Total USD)`.

In [ ]:
plt.figure(figsize=(15, 20))
sns.barplot(data=df2, y='Parent Country', x='Sales (Domestic Ultimate Total USD)')
plt.title('Horizontal Bar Plot of Sales by Parent Country')
plt.xlabel('Sales')
plt.ylabel('Parent Country')
plt.show()

From the above bar chart, we can see that different country of the parent company does affect the amount of sales.

In [ ]:
# Replace missing values in "Parent Country" with "Unknown"
df2['Parent Country'].fillna('Unknown', inplace=True)

# Get a list of parent country
parent_countries = df2['Parent Country'].unique()

# Create a list of sales for each entity type
sales_by_parent_country = [df2.loc[df2['Parent Country'] == parent_country, 'Sales (Domestic Ultimate Total USD)'].values for parent_country in parent_countries]

# Perform ANOVA test
f_val, p_val = stats.f_oneway(*sales_by_parent_country)
print(f"F-value: {f_val}, p-value: {p_val}")

The F-value of 6.950732217639041 is a measure of how much the means of `Sales (Domestic Ultimate Total USD)` for each `Parent Country` vary between the groups compared to how much they vary within the groups. A high F-value indicates that the means are significantly different.

* **2.3.5 `Employees Domestic`**


In [ ]:
sns.scatterplot(x='Employees (Domestic Ultimate Total)', y='Sales (Domestic Ultimate Total USD)', data=df)

From here, we notice that there is a negative correlation between domestic sales and the total number of individuals employed by a company and hence, we will include this feature variable.

* **2.3.6 `Sales Global`**


In [ ]:
filtered_df = df[df['Sales (Global Ultimate Total USD)'] != df['Sales (Domestic Ultimate Total USD)']]
sns.scatterplot(x='Sales (Global Ultimate Total USD)', y='Sales (Domestic Ultimate Total USD)', data=filtered_df)

From here, we notice that there is a negative correlation between domestic sales and global sales and hence, we will include this feature variable.

* **2.3.7 `Is Domestic Ultimate`**



In [ ]:
sns.boxplot(x='Is Domestic Ultimate', y='Sales (Domestic Ultimate Total USD)', data=df)
plt.xlabel('Is Domestic Ultimate')
plt.ylabel('Sales (Domestic Ultimate Total USD)')

From here, we notice that there is a relationship between `Is Domestic Ultimate` and `Sales (Domestic Ultimate Total USD)`. When company is the ultimate or highest-level company within a corporate structure based in its home country, domestic sales tend to be slightly higher on average.

### **Part 3: Model Selection**

We chose XGBoost as our predictive modeling tool for several compelling reasons. XGBoost, which stands for eXtreme Gradient Boosting, is renowned for its efficiency and effectiveness in solving classification and regression problems.

 * Performance: XGBoost is known for delivering high-performance models. It has been the algorithm behind many winning solutions in Kaggle competitions, which attests to its capability in handling a wide range of data science problems effectively.

 * Handling of Large Datasets: XGBoost is optimized to efficiently handle large datasets. Its scalability in both speed and performance is a critical factor, especially in a datathon setting where time and resources are limited.

 * Versatility: This model can handle a variety of data types, relationships, and distributions, making it a versatile choice for our diverse datasets. It also works well with both categorical and numerical inputs, which are common in many real-world datasets.

 * Robustness to Overfitting: XGBoost includes built-in regularization, which helps prevent overfitting. This is crucial in ensuring that our model generalizes well to unseen data.

 * Feature Importance: XGBoost provides useful insights into the importance of each feature in the training process, allowing us to understand which factors are driving the predictions. This interpretability is valuable in making data-driven decisions and presentations.

In sum, XGBoost offers a powerful combination of speed, performance, and flexibility, making it an excellent choice for our datathon's predictive modeling tasks.

In [ ]:
# import scikit learn packages
from sklearn.model_selection import train_test_split, cross_val_score, KFold
# import xgboost
import xgboost as xgb

### **Part 4: Model Training**

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Extract features (X) and target variable (y)
X = df.drop(columns=['Sales (Domestic Ultimate Total USD)'])
y = df['Sales (Domestic Ultimate Total USD)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
model = XGBRegressor(enable_categorical=True)

# Define hyperparameters grid for grid search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_child_weight': [1, 3, 5],
    'gamma': [0.0, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Make predictions on the testing data
y_pred = best_model.predict(X_test)

In [ ]:
# Save the model
best_model.save_model('7UP_xgboost_model.json')

### **Part 5: Model Evaluation**

To evaluate our model, we will be using evaluation metrics such as Root Mean Squared Error (RMSE), and R-squared value.

RMSE is the average of the squared differences between the predicted and actual values.

R-squared Value is the proportion of the variance in the target variable explained by the model.

In [ ]:
# Calculate the Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", rmse)

#Calculate the R-squared Value
r_squared = r2_score(y_test, y_pred)
print("R-squared:", r_squared)

From our model, we managed to obtain an RMSE of 9176994.121846402 and R-squared value of 0.6343294375554753 which is an indication that the model is decently accurate in predicting the actual value.

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [3]:
# Import relevant packages
import pandas as pd
from xgboost import XGBRegressor

def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''

    ## Load Model Function
    def load_model(filename):
        # Load the model
        loaded_model = XGBRegressor()
        loaded_model.load_model('7UP_xgboost_model.json')
        return loaded_model

    ## Preprocessing Function

    def preprocess(data: pd.DataFrame):

        ## One-hot encoding
        data = pd.get_dummies(data, columns=['Entity Type'], prefix = 'Entity Type')
        data = pd.get_dummies(data, columns=['Parent Country'], prefix='Parent Country')

        ## SIC Code handling

        # Find the frequency of each industry
        sic_code_frequency = df['SIC Code'].value_counts()
        # Note, use original df here so the same list of SIC codes are used as in the xgboost model.

        # Set a threshold for low-frequency SIC Codes
        threshold = 7  # Adjust this threshold based on your preference

        # Identify SIC Codes with frequency below the threshold
        low_frequency_sic_codes = sic_code_frequency[sic_code_frequency < threshold].index

        # Replace these low-frequency SIC Codes with a common label "Others"
        data['SIC Code'] = data['SIC Code'].replace(low_frequency_sic_codes, 'Others')

        sic_code_frequency1 = data['SIC Code'].value_counts()

        # Display the updated DataFrame
        print(sic_code_frequency1)

        # Convert SIC Code to a 'category' data type so that it can be read by xgboost
        data['SIC Code'] = data['SIC Code'].astype('category')

        ## Column removal
        columns_to_remove = ["AccountID", "Company", "8-Digit SIC Code", "8-Digit SIC Description", "Industry", "Company Description", "Company Status (Active/Inactive)", "Ownership Type", "Square Footage", "Parent Company", "Domestic Ultimate Company", "Global Ultimate Company", "Year Found", "Fiscal Year End", "Employees (Single Site)", "Employees (Global Ultimate Total)", "Import/Export Status", "Global Ultimate Country", "Is Global Ultimate"]
        data = data.drop(columns=columns_to_remove)

        ## Latitude or Longitude NA value handling

        # Remove rows without either Latitude or Longitude coordinates
        data = data.dropna(subset=["LATITUDE", "LONGITUDE"])
        data = data.dropna(subset=["Employees (Domestic Ultimate Total)"])

        return data


    # Preprocess data
    hidden_data = preprocess(hidden_data)

    # Load model
    model = load_model

    # Make predictions
    predictions = model.predict(hidden_data)

    # Convert to list
    result = predictions.tolist()

    return result

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_csv(filepath)
test_df = test_df.drop(columns=['Sales (Domestic Ultimate Total USD)'])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!